## 1. Dependancy Installation

In [1]:
# %pip install tensorflow_version 1.x
# %pip install --upgrade h5py==2.10.0
# %pip install opencv-python
import sys
sys.path.append("../Mask_RCNN/mrcnn")
from m_rcnn import *
from visualize import random_colors, get_mask_contours, draw_mask
%matplotlib inline

VERS 0.2


Using TensorFlow backend.


## 2. Run Mask-RCNN on Images


In [2]:
# Load Image

train_annotations_path = "../dataset/lc_dataset_v6/train.json"
val_annotations_path = "../dataset/lc_dataset_v6/test.json"

dataset_train = load_image_dataset(os.path.join( train_annotations_path), "../dataset/lc_dataset_v6/train", "train")
dataset_val = load_image_dataset(os.path.join( val_annotations_path), "../dataset/lc_dataset_v6/test", "test")
class_number = dataset_train.count_classes()

print("Classes: {}".format(class_number))

Annotation json path:  ../dataset/lc_dataset_v6/train.json
Annotation json path:  ../dataset/lc_dataset_v6/test.json
Train: 240
Train: 7
Classes: 1


In [3]:

img = cv2.imread("./output/MasDetection.png")


test_model, inference_config = load_inference_model(1, "../models/lung_cancer/lung_cancer_model_v6.h5")
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect results
r = test_model.detect([image])[0]
colors = random_colors(80)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from  ../models/lung_cancer/lung_cancer_model_v6.h5



In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Get Coordinates and show it on the image
object_count = len(r["class_ids"])
for i in range(object_count):
    # 1. Mask
    mask = r["masks"][:, :, i]
    contours = get_mask_contours(mask)
    for cnt in contours:
        cv2.polylines(img, [cnt], True, colors[i], 2)
        img = draw_mask(img, [cnt], colors[i])
# cv2.Waitkey(10000)
# cv2.imshow("img",img)
plt.imsave("step1_lc_output.png", img)
print(image)

[[[253 231  36]
  [253 231  36]
  [253 231  36]
  ...
  [253 231  36]
  [253 231  36]
  [253 231  36]]

 [[253 231  36]
  [253 231  36]
  [253 231  36]
  ...
  [253 231  36]
  [253 231  36]
  [253 231  36]]

 [[253 231  36]
  [253 231  36]
  [253 231  36]
  ...
  [253 231  36]
  [253 231  36]
  [253 231  36]]

 ...

 [[ 68   1  84]
  [ 68   1  84]
  [ 68   1  84]
  ...
  [ 32 144 140]
  [253 231  36]
  [253 231  36]]

 [[ 68   1  84]
  [ 68   1  84]
  [ 68   1  84]
  ...
  [253 231  36]
  [253 231  36]
  [253 231  36]]

 [[ 68   1  84]
  [ 68   1  84]
  [ 68   1  84]
  ...
  [253 231  36]
  [253 231  36]
  [253 231  36]]]


## 3. Model Summary

In [5]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 30 images. Increase for better accuracy.
from model_accuracy import *

model_accuracy(dataset_train, dataset_val, test_model, inference_config, modellib,utils)

Model Accuracy:  0.7697222235467699


In [6]:
print(test_model.keras_model.summary())

Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________